In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from prophet import Prophet
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
from datetime import timedelta, datetime
import random

def load_and_clean_data(file_path):
    data = pd.read_excel(file_path, parse_dates=['DateAndHour'])
    data['Hour'] = data['DateAndHour'].dt.hour
    data['DayOfWeek'] = data['DateAndHour'].dt.dayofweek
    data['Month'] = data['DateAndHour'].dt.month
    data = data.dropna(subset=['Temperature', 'Load_data'])
    return data

def add_lag_and_rolling_features(data, lags=[1, 24, 168], window=3):
    for lag in lags:
        data[f'Load_data_lag_{lag}'] = data['Load_data'].shift(lag)
        data[f'Temperature_lag_{lag}'] = data['Temperature'].shift(lag)
    
    data[f'Load_data_rolling_{window}'] = data['Load_data'].rolling(window=window).mean()
    data[f'Load_data_rolling_std_{window}'] = data['Load_data'].rolling(window=window).std()
    data['Temperature_change'] = data['Temperature'].diff()
    
    data['Temperature_bins'] = pd.cut(data['Temperature'], bins=[-np.inf, 0, 15, np.inf], labels=['cold', 'moderate', 'hot'])
    data = pd.get_dummies(data, columns=['Temperature_bins'], drop_first=True)
    
    return data

def prepare_data_for_date(data, target_date, features):
    if isinstance(target_date, str):
        target_date = datetime.strptime(target_date, '%d/%m/%Y').date()
    
    if target_date in data['DateAndHour'].dt.date.values:
        target_data = data[data['DateAndHour'].dt.date == target_date].copy()
    else:
        # Create a DataFrame for the future date
        target_data = pd.DataFrame({
            'DateAndHour': pd.date_range(start=target_date, periods=24, freq='H'),
            'Hour': range(24),
            'DayOfWeek': target_date.weekday(),
            'Month': target_date.month
        })
        # Use average temperature from historical data for the same month
        avg_temp_by_hour = data[data['DateAndHour'].dt.month == target_date.month].groupby(data['DateAndHour'].dt.hour)['Temperature'].mean()
        target_data['Temperature'] = target_data['Hour'].map(avg_temp_by_hour)
    
    for feature in features:
        if feature not in target_data.columns:
            target_data[feature] = 0
    
    return target_data

def debug_data(data, name):
    print(f"\nDebugging {name}:")
    print(f"Type: {type(data)}")
    print(f"Shape: {data.shape}")
    
    if isinstance(data, pd.DataFrame):
        print("Columns:")
        print(data.columns)
        print("\nFirst few rows:")
        print(data.head())
        print("\nData types:")
        print(data.dtypes)
        print("\nMissing values:")
        print(data.isnull().sum())
    elif isinstance(data, pd.Series):
        print("Name:", data.name)
        print("\nFirst few values:")
        print(data.head())
        print("\nData type:")
        print(data.dtype)
        print("\nMissing values:")
        print(data.isnull().sum())
    
    print("\nSummary statistics:")
    print(data.describe())

def xgboost_model(X_train, y_train, X_future):
    model = xgb.XGBRegressor(n_estimators=200, learning_rate=0.05, max_depth=6, random_state=42)
    model.fit(X_train, y_train)
    return model.predict(X_future)

def prophet_model(data_train, future):
    model = Prophet(daily_seasonality=True)
    model.add_regressor('Temperature')
    model.fit(data_train)
    forecast = model.predict(future)
    return forecast['yhat'].values

def arima_model(y_train, n_future):
    model = ARIMA(y_train, order=(5,1,2))
    results = model.fit()
    return results.forecast(steps=n_future)

def random_forest_model(X_train, y_train, X_future):
    model = RandomForestRegressor(n_estimators=200, max_depth=15, min_samples_split=5, random_state=42)
    model.fit(X_train, y_train)
    return model.predict(X_future)

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero = (y_true != 0)
    return np.mean(np.abs((y_true[non_zero] - y_pred[non_zero]) / y_true[non_zero])) * 100

# Main execution
if __name__ == "__main__":
    # Load and prepare data
    data = load_and_clean_data('Hackathon_Data_Cleaned.xlsx')
    
    # Define lags
    lags = [1, 24, 168]
    
    data = add_lag_and_rolling_features(data, lags=lags, window=3)

    debug_data(data, "Historical Data")

    # Define features
    features = ['Hour', 'DayOfWeek', 'Month', 'Temperature', 'Temperature_change',
                'Load_data_lag_1', 'Load_data_lag_24', 'Load_data_lag_168', 
                'Temperature_lag_1', 'Temperature_lag_24', 'Temperature_lag_168',
                'Load_data_rolling_3', 'Load_data_rolling_std_3',
                'Temperature_bins_moderate', 'Temperature_bins_hot']

    # Prepare training data
    X_train = data[features].dropna()
    y_train = data['Load_data'].loc[X_train.index]

#     debug_data(X_train, "Training Features")
#     debug_data(y_train, "Training Target")

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)

    # Select random dates for prediction (excluding the last week to ensure we have actual data for comparison)
    max_date = data['DateAndHour'].dt.date.max() - timedelta(days=7)
    min_date = data['DateAndHour'].dt.date.min() + timedelta(days=max(lags))
    available_dates = pd.date_range(start=min_date, end=max_date).date
    random_dates = random.sample(list(available_dates), 2)  # Select 5 random dates

    # Add manual dates
    manual_dates = ['26/03/2024', '30/03/2024']
    all_dates = random_dates + manual_dates

    all_predictions = []
    all_actuals = []

    for target_date in all_dates:
        print(f"\nPredicting for date: {target_date}")
        
        # Prepare data for the target date
        target_data = prepare_data_for_date(data, target_date, features + ['DateAndHour'])
        X_target_scaled = pd.DataFrame(scaler.transform(target_data[features]), columns=features, index=target_data.index)
        
        # Generate predictions
        xgb_pred = xgboost_model(X_train_scaled, y_train, X_target_scaled)
        
        prophet_data_train = data[['DateAndHour', 'Load_data', 'Temperature']].rename(columns={'DateAndHour': 'ds', 'Load_data': 'y'})
        prophet_target = target_data[['DateAndHour', 'Temperature']].rename(columns={'DateAndHour': 'ds'})
        prophet_pred = prophet_model(prophet_data_train, prophet_target)
        
        rf_pred = random_forest_model(X_train_scaled, y_train, X_target_scaled)
        arima_pred = arima_model(y_train, len(target_data))
        
        # Ensemble prediction
        final_pred = np.nanmean([xgb_pred, prophet_pred, rf_pred, arima_pred], axis=0)
        
        # For random dates, calculate MAPE
#         if target_date not in manual_dates:
        actual_values = data[data['DateAndHour'].dt.date == target_date]['Load_data'].values
        all_predictions.extend(final_pred)
        all_actuals.extend(actual_values)
        mape = mean_absolute_percentage_error(actual_values, final_pred)
        print(f"MAPE for {target_date}: {mape:.2f}%")
#         else:
#             print(f"Predictions for {target_date}:")
#             for hour, pred in enumerate(final_pred):
#                 print(f"Hour {hour}: {pred:.2f}")
        
        # Plotting
        plt.figure(figsize=(12, 6))
        plt.plot(range(24), final_pred, label='Predicted Load', marker='x')
        if target_date not in manual_dates:
            plt.plot(range(24), actual_values, label='Actual Load', marker='o')
        plt.title(f'Load Prediction for {target_date}')
        plt.xlabel('Hour')
        plt.ylabel('Load')
        plt.legend()
        plt.grid(True)
        plt.savefig(f'load_prediction_{target_date}.png')
        plt.close()

    # Calculate overall MAPE for random dates
    if all_actuals:
        overall_mape = mean_absolute_percentage_error(all_actuals, all_predictions)
        print(f"\nOverall MAPE for random dates: {overall_mape:.2f}%")

    print("\nPrediction plots saved as 'load_prediction_<date>.png'")